In [3]:
import pandas as pd
import importlib
import gc
import sys
sys.path.append('./scripts')  
import dataset
importlib.reload(dataset)

<module 'dataset' from 'c:\\Users\\Usuario\\Documents\\Universidad\\austral\\2025\\Lab3\\Lab3-MCD\\notebooks\\entregable\\./scripts\\dataset.py'>

In [4]:
df = pd.read_csv("../../data/preprocessed/base.csv", sep=',')
df.shape

(2945818, 13)

In [5]:
#### COMBINATORIA ####
data = dataset.combinatoria_periodo_producto()
data['periodo'] = data['periodo'].dt.year * 100 + data['periodo'].dt.month
data.shape

(44388, 2)

In [6]:
#### PRODUCTOS SIN HISTORIA ####
#### Vemos que hay  42 + 7 + 20 productos que tienen historia <= 3 meses. Los elimino.
prod_sin_historia = dataset.getProductos_sinHistoria()
data = data[~data['product_id'].isin(prod_sin_historia)]
data.shape

(41904, 2)

In [7]:
#### PRODUCTOS QUE NO NACIERON ####
data = dataset.eliminarProductos_sinNacer(df, data)
data.shape

(34945, 3)

In [8]:
#### PRODUCTOS QUE MURIERON ####
data = dataset.eliminarProductosMuertos(df, data)
data.shape

(31362, 4)

In [9]:
#### 3MESES PRODUCTOS NUEVOS: HISTORIA MENOR A 12 MESES ####
data = dataset.marcarProductosNuevos_3M(data)
data.shape

(31362, 8)

In [10]:
data

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial
0,20001,201701,2017-01-01,2019-12-01,1,36,0,0
1,20001,201702,2017-01-01,2019-12-01,2,36,0,0
2,20001,201703,2017-01-01,2019-12-01,3,36,0,0
3,20001,201704,2017-01-01,2019-12-01,4,36,0,0
4,20001,201705,2017-01-01,2019-12-01,5,36,0,0
...,...,...,...,...,...,...,...,...
31357,21281,201704,2017-02-01,2017-08-01,3,7,1,1
31358,21281,201705,2017-02-01,2017-08-01,4,7,1,0
31359,21281,201706,2017-02-01,2017-08-01,5,7,1,0
31360,21281,201707,2017-02-01,2017-08-01,6,7,1,0


In [11]:
#### MERGE CON PRODUCTOS ####
productos = pd.read_csv("../../data/raw/tb_productos.csv", sep='\t')
productos = productos.drop_duplicates(subset=['product_id'], keep='first')
data = data.merge(productos, how='left', on="product_id")
del productos

#### MERGE CON STOCKS ####
stocks = pd.read_csv("../../data/raw/tb_stocks.csv", sep='\t')
stocks = stocks.groupby(by=["periodo", "product_id"]).agg({"stock_final": "sum"}).reset_index()
data = data.merge(stocks, how='left', on=['periodo', 'product_id'])
del stocks

#### MERGE CON SELLIN ####
sellin = pd.read_csv("../../data/raw/sell-in.csv", sep='\t')
sellin = sellin.groupby(by=["periodo","product_id"]).agg({"tn":"sum", "plan_precios_cuidados":"sum", "cust_request_qty":"sum", "cust_request_tn":"sum"}).reset_index()
data = data.merge(sellin, how='left', on=['periodo', 'product_id'])
del sellin
gc.collect()

0

In [12]:
data

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,cat1,cat2,cat3,brand,sku_size,stock_final,tn,plan_precios_cuidados,cust_request_qty,cust_request_tn
0,20001,201701,2017-01-01,2019-12-01,1,36,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,934.77222,0.0,479.0,937.72717
1,20001,201702,2017-01-01,2019-12-01,2,36,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,798.01620,0.0,432.0,833.72187
2,20001,201703,2017-01-01,2019-12-01,3,36,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,1303.35771,0.0,509.0,1330.74697
3,20001,201704,2017-01-01,2019-12-01,4,36,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,1069.96130,0.0,279.0,1132.94430
4,20001,201705,2017-01-01,2019-12-01,5,36,0,0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,1502.20132,0.0,701.0,1550.68936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31357,21281,201704,2017-02-01,2017-08-01,3,7,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31358,21281,201705,2017-02-01,2017-08-01,4,7,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31359,21281,201706,2017-02-01,2017-08-01,5,7,1,0,NaN,NaN,NaN,NaN,NaN,NaN,0.09134,0.0,8.0,0.10539
31360,21281,201707,2017-02-01,2017-08-01,6,7,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#### COMPLETO TN CON CEROS ####
####  ¿cuantos?
print(f"Total de periodos con Nan debido a la combinatoria periodo_x_producto: {data['tn'].isna().sum()}")
#### Lo completo con ceros
data['tn'] = data['tn'].fillna(0)

Total de periodos con Nan debido a la combinatoria periodo_x_producto: 276


In [14]:
#### GUARDAR DATAFRAME ####
data["nacimiento_producto"] = data["nacimiento_producto"].dt.year * 100 + data["nacimiento_producto"].dt.month
data["muerte_producto"] = data["muerte_producto"].dt.year * 100 + data["muerte_producto"].dt.month
data[['nacimiento_producto','muerte_producto']]

,nacimiento_producto,muerte_producto
0,201701,201912
1,201701,201912
2,201701,201912
3,201701,201912
4,201701,201912
...,...,...
31357,201702,201708
31358,201702,201708
31359,201702,201708
31360,201702,201708


In [15]:
data.to_csv("./datasets/periodo_x_producto.csv", index=False, sep=',', encoding='utf-8')